In [1]:
import warnings
warnings.filterwarnings('ignore')

### Wandb session

In [2]:
import wandb
wandb.login(key='6f62d15846ae3ef63e0c80b3ab1f4de94d5cc8a4')
wandb.init(project='augmented_test')

import os
os.environ["WANDB_DISABLED"] = "true"


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: metalwallcrusher (metalwallcrusher-vinuniversity) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Tracking run with wandb version 0.19.6
wandb: Run data is saved locally in /kaggle/working/wandb/run-20250414_072506-l0t2t29t
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run major-salad-88
wandb: ⭐️ View project at https://wandb.ai/metalwallcrusher-vinuniversity/augmented_test
wandb: 🚀 View run at https://wandb.ai/metalwallcrusher-vinuniversity/augmented_test/runs/l0t2t29t


### Cleaning the kaggle/working directory

In [3]:
import os
import shutil

directory = '/kaggle/working'

# Loop through each item in the directory
for filename in os.listdir(directory):
    file_path = os.path.join(directory, filename)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)  # Removes files and links
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)  # Removes directories and their contents
    except Exception as e:
        print(f'Failed to delete {file_path}. Reason: {e}')

print("Cleaning complete.")

Cleaning complete.


### Installing dependencies

In [4]:
!pip install -q -U sentencepiece
!pip install -q -U datasets
!pip install -q -U sacrebleu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 4.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.12.0 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.8.4.1 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cudnn-cu12==9.1.0.70; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cudnn-cu12 9.3.0.75 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cufft-cu12==11.2.1.3; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cufft-cu12 11.3.3.83 which is incompatible.
torch 2.5.1+cu124 requires nvidia-curand-cu12==10.3.5.147; platform_system == "Linux" and platform_machin

### Importing libraries

In [5]:
from datasets import Dataset, concatenate_datasets

In [6]:
import torch
import sentencepiece
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainer, Seq2SeqTrainingArguments
from datasets import load_dataset
# import numpy as np 
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset, concatenate_datasets

import sacrebleu

2025-04-14 07:25:36.020237: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744615536.208768      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744615536.268290      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


### Reproducibility

In [7]:
RANDOM_SEED = 42
# np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
# If you are using CUDA
torch.cuda.manual_seed_all(RANDOM_SEED)
# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

MAX_LEN = 384
PADDING_LEN = 228

### Loading the pretrained model

In [8]:
model_name = "Helsinki-NLP/opus-mt-vi-en"  # Example model for Bahnar to English
ba_en_tokenizer = AutoTokenizer.from_pretrained("/kaggle/input/update-en-vi-ba-tokenizer")
vi_en_tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/756k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/809k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.19M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/289M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

### Loading the dataset

In [9]:
with open('/kaggle/input/train-eng-bdq-phase2dataset-v1/bdq-eng.train.bdq', 'r', encoding='utf-8') as file_ba:
    ba_data = file_ba.readlines()
with open('/kaggle/input/train-eng-bdq-phase2dataset-v1/bdq-eng.train.eng', 'r', encoding='utf-8') as file_en:
    en_data = file_en.readlines()
assert len(ba_data) == len(en_data), "The files don't have the same number of lines."
train_df = pd.DataFrame({'English': en_data, 'Bahnar': ba_data})
train_df, val_df = train_test_split(train_df, test_size=0.1, random_state=RANDOM_SEED)

ba_en_train_dataset = Dataset.from_pandas(train_df)
ba_en_val_dataset = Dataset.from_pandas(val_df)

with open('/kaggle/input/train-vie-eng-phomt-dev-dataset/dev.vi', 'r', encoding='utf-8') as file_vi:
    vi_data = file_vi.readlines()
with open('/kaggle/input/train-vie-eng-phomt-dev-dataset/dev.en', 'r', encoding='utf-8') as file_en:
    en_data = file_en.readlines()
assert len(vi_data) == len(en_data), "The files don't have the same number of lines."
train_df = pd.DataFrame({'English': en_data, 'Vietnamese': vi_data})

vi_en_train_dataset = Dataset.from_pandas(train_df)

model.safetensors:   0%|          | 0.00/289M [00:00<?, ?B/s]

In [10]:

with open('/kaggle/input/test-eng-bdq-phase2dataset-v1/bdq-eng.test.bdq', 'r', encoding='utf-8') as file_ba:
    ba_data = file_ba.readlines()
with open('/kaggle/input/test-eng-bdq-phase2dataset-v1/bdq-eng.test.eng', 'r', encoding='utf-8') as file_en:
    en_data = file_en.readlines()
assert len(ba_data) == len(en_data), "The files don't have the same number of lines."
test_df = pd.DataFrame({'English': en_data, 'Bahnar': ba_data})
ba_en_test_dataset = Dataset.from_pandas(test_df)

print(len(ba_en_train_dataset), len(ba_en_val_dataset), len(ba_en_test_dataset))
print(len(vi_en_train_dataset))

21801 2423 1000
18719


### Adding more pivot, translated data


In [11]:
#Add pivot
# with open('/kaggle/input/enviba/train.ba', 'r', encoding='utf-8') as file_ba:
#     ba_data = file_ba.readlines()
# with open('/kaggle/input/enviba/train2.en', 'r', encoding='utf-8') as file_en:
#     en_data = file_en.readlines()
# assert len(ba_data) == len(en_data), "The files don't have the same number of lines."
# aug_train_df = pd.DataFrame({'English': en_data, 'Bahnar': ba_data})

# augmented = Dataset.from_pandas(aug_train_df)
# ba_en_train_dataset = concatenate_datasets([ba_en_train_dataset, augmented]).shuffle(seed=RANDOM_SEED)

#### Investigate current tokenizer

In [12]:
len(ba_en_tokenizer.encoder)

53739

In [13]:
with ba_en_tokenizer.as_target_tokenizer():
    print(len(ba_en_tokenizer.encoder))

53739


In [14]:

print(ba_en_train_dataset[0]["Bahnar"])
print(ba_en_tokenizer.tokenize(ba_en_train_dataset[0]["Bahnar"]))
list(ba_en_tokenizer.encode(ba_en_train_dataset[0]["Bahnar"]))

Pơhrôp pơguăng chô̆ ăn maih tơ ao lu iĕm

['Pơ', '▁h', 'r', 'ôp', 'pơ', '▁gu', 'ăng', '▁ch', 'ô̆', '▁ăn', '▁ma', 'ih', '▁tơ', '▁ao', '▁lu', 'iĕ', '▁m']


[56013,
 3654,
 1067,
 55285,
 53993,
 10810,
 19469,
 4257,
 58366,
 317,
 870,
 57376,
 3063,
 5908,
 6741,
 55393,
 2636,
 0]

In [15]:

print(ba_en_train_dataset[0]["Bahnar"])
print(vi_en_tokenizer.tokenize(ba_en_train_dataset[0]["Bahnar"]))
list(vi_en_tokenizer.encode(ba_en_train_dataset[0]["Bahnar"]))

Pơhrôp pơguăng chô̆ ăn maih tơ ao lu iĕm

['▁P', 'ơ', 'h', 'rô', 'p', '▁p', 'ơ', 'gu', 'ăng', '▁chô', '̆', '▁ăn', '▁mai', 'h', '▁tơ', '▁ao', '▁lu', '▁', 'i', 'ĕ', 'm']


[1929,
 15619,
 1266,
 7008,
 883,
 3843,
 15619,
 5782,
 19469,
 7751,
 49915,
 317,
 1396,
 1266,
 3063,
 5908,
 6741,
 301,
 231,
 1,
 78,
 0]

In [16]:
print(ba_en_train_dataset[0]["English"])
print(ba_en_tokenizer.tokenize(ba_en_train_dataset[0]["English"]))
list(ba_en_tokenizer.encode(ba_en_train_dataset[0]["English"]))

Who put on ornaments of gold upon your apparel

['▁Who', '▁put', '▁on', '▁or', 'na', 'ments', '▁of', '▁', 'gold', '▁', 'upon', '▁your', '▁app', 'are', 'l']


[654,
 578,
 55,
 196,
 1344,
 10998,
 12,
 301,
 46185,
 301,
 25166,
 77,
 12811,
 7885,
 1194,
 0]

In [17]:
print(ba_en_train_dataset[0]["English"])
print(vi_en_tokenizer.tokenize(ba_en_train_dataset[0]["English"]))
list(vi_en_tokenizer.encode(ba_en_train_dataset[0]["English"]))

Who put on ornaments of gold upon your apparel

['▁Who', '▁put', '▁on', '▁or', 'na', 'ments', '▁of', '▁', 'gold', '▁', 'upon', '▁your', '▁app', 'are', 'l']


[654,
 578,
 55,
 196,
 1344,
 10998,
 12,
 301,
 46185,
 301,
 25166,
 77,
 12811,
 7885,
 1194,
 0]

### Preprocess the dataset with tokenizer

In [20]:
def preprocess_function_target(examples):
    inputs = examples['Bahnar']
    targets = examples['English']
    try:
        model_inputs = ba_en_tokenizer(inputs, max_length=PADDING_LEN, padding='max_length', truncation=True)
    
        # Set up the tokenizer for Bahnar
        with ba_en_tokenizer.as_target_tokenizer():
            labels = ba_en_tokenizer(targets, max_length=PADDING_LEN, padding='max_length', truncation=True)
    
        model_inputs["labels"] = labels["input_ids"]
        return model_inputs
    except:
        print(inputs)
    

tokenized_ba_en = ba_en_train_dataset.map(preprocess_function_target, batched=True)
tokenized_ba_en_val = ba_en_val_dataset.map(preprocess_function_target, batched=True)

Map:   0%|          | 0/21801 [00:00<?, ? examples/s]

Map:   0%|          | 0/2423 [00:00<?, ? examples/s]

In [21]:
def preprocess_function_source(examples):
    inputs = examples['Vietnamese']
    targets = examples['English']
    model_inputs = vi_en_tokenizer(inputs, max_length=PADDING_LEN, padding='max_length', truncation=True)

    # Set up the tokenizer for Vietnamese
    with vi_en_tokenizer.as_target_tokenizer():
        labels = vi_en_tokenizer(targets, max_length=PADDING_LEN, padding='max_length', truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_vi_en = vi_en_train_dataset.map(preprocess_function_source, batched=True)

Map:   0%|          | 0/18719 [00:00<?, ? examples/s]

### Concatenate Vietnamese-English and Bahnar-English Dataset

In [22]:
tokenized_viAndba_en = concatenate_datasets([tokenized_vi_en, tokenized_ba_en ])
# viOrba_en_dataset is now a combined dataset
print(tokenized_viAndba_en)

Dataset({
    features: ['English', 'Vietnamese', 'input_ids', 'attention_mask', 'labels', 'Bahnar', '__index_level_0__'],
    num_rows: 40520
})


In [23]:
# shuffled_dataset = tokenized_viAndba_en.shuffle(seed=RANDOM_SEED)


### Train the model

#### Train the embedding

In [24]:
# add new, random embeddings for the new tokens
model.resize_token_embeddings(len(ba_en_tokenizer))

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Embedding(58879, 512, padding_idx=53738)

In [25]:
# Freeze all layers except for the embedding layers
def freeze_model_except_embeddings(model):
    for name, param in model.named_parameters():
        if 'embed_tokens' not in name and 'shared' not in name:
            param.requires_grad = False
        else:
            param.requires_grad = True

# Apply the function to freeze the model
freeze_model_except_embeddings(model)

In [26]:
model.to(device)

MarianMTModel(
  (model): MarianModel(
    (shared): Embedding(58879, 512, padding_idx=53738)
    (encoder): MarianEncoder(
      (embed_tokens): Embedding(58879, 512, padding_idx=53738)
      (embed_positions): MarianSinusoidalPositionalEmbedding(512, 512)
      (layers): ModuleList(
        (0-5): 6 x MarianEncoderLayer(
          (self_attn): MarianAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation_fn): SiLU()
          (fc1): Linear(in_features=512, out_features=2048, bias=True)
          (fc2): Linear(in_features=2048, out_features=512, bias=True)
          (final_layer_norm): LayerNorm((512,), eps=1e-05

In [27]:
from transformers import DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer, EarlyStoppingCallback

print((len(tokenized_viAndba_en)))
# Use data collator
data_collator = DataCollatorForSeq2Seq(ba_en_tokenizer, model=model)

40520


In [28]:
# Define training arguments for training the embedding layers
embedding_training_args = Seq2SeqTrainingArguments(
    output_dir="./results_embedding",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=3,  # Adjust the number of epochs as needed
    weight_decay=0.015,
    save_total_limit=3,
)

# Create a trainer for training the embedding layers
embedding_trainer = Seq2SeqTrainer(
    model=model,
    args=embedding_training_args,
    train_dataset=tokenized_viAndba_en,
    eval_dataset=tokenized_ba_en_val,
    data_collator=data_collator,
)

# Train the embedding layers
embedding_trainer.train()

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Epoch,Training Loss,Validation Loss
1,2.030200,0.535799
2,0.384400,0.441054
3,0.357100,0.429123


TrainOutput(global_step=3801, training_loss=1.2431512545109171, metrics={'train_runtime': 2038.5564, 'train_samples_per_second': 59.63, 'train_steps_per_second': 1.865, 'total_flos': 7339970584903680.0, 'train_loss': 1.2431512545109171, 'epoch': 3.0})

#### Train the embedding and the encoding

In [29]:
def freeze_model_except_emb_enc(model):
    for name, param in model.named_parameters():
        if not (name.startswith("model.encoder") or "embed_tokens" in name or "shared" in name):
            param.requires_grad = False
        else:
            param.requires_grad = True

model.to("cpu")
# Apply the function to freeze the model
freeze_model_except_emb_enc(model)
model.to(device)

# Define training arguments for training the embedding layers
emb_enc_training_args = Seq2SeqTrainingArguments(
    output_dir="./results_embedding",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=4,  # Adjust the number of epochs as needed
    weight_decay=0.015,
    save_total_limit=3,
)

# Create a trainer for training the embedding layers
emb_enc_trainer = Seq2SeqTrainer(
    model=model,
    args=embedding_training_args,
    train_dataset=tokenized_viAndba_en,
    eval_dataset=tokenized_ba_en_val,
    data_collator=data_collator,
)

# Train the embedding layers
emb_enc_trainer.train()


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Epoch,Training Loss,Validation Loss
1,0.268900,0.310849
2,0.223400,0.272432
3,0.213300,0.263793


TrainOutput(global_step=3801, training_loss=0.238301370244878, metrics={'train_runtime': 2209.3867, 'train_samples_per_second': 55.02, 'train_steps_per_second': 1.72, 'total_flos': 7339970584903680.0, 'train_loss': 0.238301370244878, 'epoch': 3.0})

#### Train the whole model

In [30]:
# Unfreeze all layers
def unfreeze_model(model):
    for param in model.parameters():
        param.requires_grad = True

# Apply the function to unfreeze the model
model.to("cpu")
unfreeze_model(model)
model.to(device)

MarianMTModel(
  (model): MarianModel(
    (shared): Embedding(58879, 512, padding_idx=53738)
    (encoder): MarianEncoder(
      (embed_tokens): Embedding(58879, 512, padding_idx=53738)
      (embed_positions): MarianSinusoidalPositionalEmbedding(512, 512)
      (layers): ModuleList(
        (0-5): 6 x MarianEncoderLayer(
          (self_attn): MarianAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation_fn): SiLU()
          (fc1): Linear(in_features=512, out_features=2048, bias=True)
          (fc2): Linear(in_features=2048, out_features=512, bias=True)
          (final_layer_norm): LayerNorm((512,), eps=1e-05

In [31]:
# Define training arguments for training the entire model
full_training_args = Seq2SeqTrainingArguments(
    output_dir="./results_full",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=4,  # Adjust the number of epochs as needed
    weight_decay=0.015,
    save_total_limit=3,
)

# Create a trainer for training the entire model
full_trainer = Seq2SeqTrainer(
    model=model,
    args=full_training_args,
    train_dataset=tokenized_viAndba_en,
    eval_dataset=tokenized_ba_en_val,
    data_collator=data_collator,
)

# Train the entire model
full_trainer.train()

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Epoch,Training Loss,Validation Loss
1,0.193600,0.218288
2,0.168000,0.204004
3,0.159500,0.195845
4,0.152100,0.193768


TrainOutput(global_step=5068, training_loss=0.16817206798243353, metrics={'train_runtime': 3259.2091, 'train_samples_per_second': 49.73, 'train_steps_per_second': 1.555, 'total_flos': 9786627446538240.0, 'train_loss': 0.16817206798243353, 'epoch': 4.0})

### Test the model

In [32]:
import torch
from tqdm.auto import tqdm

def evaluate_model(model, encode_tokenizer, decode_tokenizer, dataset, device):
    model.eval()
    prt = True

    predictions, references = [], []
    for example in tqdm(dataset, desc="Translating"):
        with encode_tokenizer.as_target_tokenizer():
            input_ids = encode_tokenizer.encode(example['Bahnar'], return_tensors='pt').to(device)

        output_ids = model.generate(input_ids)[0]
        pred = decode_tokenizer.decode(output_ids, skip_special_tokens=True)
        predictions.append([pred])

        references.append([example['English']])

    return predictions, references


In [33]:
test_examples = [{'Bahnar': ex['Bahnar'], 'English': ex['English']} for ex in ba_en_test_dataset]
predictions, references = evaluate_model(model, ba_en_tokenizer, ba_en_tokenizer, test_examples, device)

Translating:   0%|          | 0/1000 [00:00<?, ?it/s]

In [34]:
predictions[0], references[0] # Check format of predictions and references

(['Tho u shalt do no more than that, thou shalt do no more'],
 ['I release you, but can do no more.\n'])

In [35]:
len(predictions), len(references)

(1000, 1000)

#### Calculate sacreBLEU

In [36]:
preds = []
refs = []
for pred in predictions:
    preds.append(pred[0])

for ref in references:
    refs.append(ref)

print(refs[:2])
print(preds[:2])

[['I release you, but can do no more.\n'], ['You have as much right to be a knight as any man.\n']]
['Tho u shalt do no more than that, thou shalt do no more', 'Tho u art not a man of understanding']


In [37]:
# Calculate and print the BLEU score
bleu = sacrebleu.corpus_bleu(preds, refs)
print("BLEU: ", round(bleu.score, 2))

# Calculate CHRF
chrf = sacrebleu.corpus_chrf(preds, refs)
print("CHRF:", round(chrf.score, 2))

# Calculate TER
metric = sacrebleu.metrics.TER()
ter = metric.corpus_score(preds, refs)
print("TER:", round(ter.score, 2))

# wandb.log({"BLEU": round(bleu.score, 2), "CHRF": round(chrf.score, 2), "TER": round(ter.score, 2)})


BLEU:  21.36
CHRF: 27.89
TER: 80.01


### Saving the model

In [38]:
# Step 5: Save the Model
model.save_pretrained("./model1.1.1")
ba_en_tokenizer.save_pretrained("./tokenizer1.1.1")
vi_en_tokenizer.save_pretrained("./tokenizer")

('./tokenizer/tokenizer_config.json',
 './tokenizer/special_tokens_map.json',
 './tokenizer/vocab.json',
 './tokenizer/source.spm',
 './tokenizer/target.spm',
 './tokenizer/added_tokens.json')

### Display translated and references for qualitative evaluation

In [39]:
# Display the first 3 prediction-reference pairs
for i in range(10):
    print(f"Prediction {i+1}: {predictions[i][0]}")
    print(f"Reference {i+1}: {references[i][0]}")

Prediction 1: Tho u shalt do no more than that, thou shalt do no more
Reference 1: I release you, but can do no more.

Prediction 2: Tho u art not a man of understanding
Reference 2: You have as much right to be a knight as any man.

Prediction 3: No, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no.
Reference 3: No. Leave me alone.

Prediction 4: Where are you going to help me?
Reference 4: I'm here to help!

Prediction 5: And all t hin gs that thou hast done are good and bad: and all t hin gs that thou hast done are good and bad; and all t hin gs that thou hast done are good and bad
Reference 5: All you wanna do is pee.

Prediction 6: And it shall come to pass, that there shall be no end of all t hin gs to do in all the service of the land, and in all the land of Egypt, and in all the multitude thereof, and in all the multitude thereof, and in all the multitude thereof
Reference 6: And these platforms were certainly very helpful to activists in Tunisia and Egypt this pa

### Clean up the output

In [40]:

# import os
# import shutil

# def clear_directory_except_subdir(directory, subdir):
#     for item in os.listdir(directory):
#         item_path = os.path.join(directory, item)
#         if os.path.isdir(item_path) and item_path not in subdir:
#             shutil.rmtree(item_path)  # Remove directories
#         elif os.path.isfile(item_path) and item_path not in subdir:
#             os.remove(item_path)  # Remove files

# # Usage
# working_dir = '/kaggle/working'
# results_dir = ['/kaggle/working/results', '/kaggle/working/en-ba-model', '/kaggle/working/en-ba-tokenizer', '/kaggle/working/en-ba-aug-tokenizer', '/kaggle/working/en-ba-aug-model']
# clear_directory_except_subdir(working_dir, results_dir)


# Zip the model

In [41]:
# !zip -r model1.1.1.zip /kaggle/working/model1.1.1

In [42]:
# !zip -r tokenizer1.1.1.zip /kaggle/working/tokenizer1.1.1

In [43]:
# !zip -r tokenizer.zip /kaggle/working/tokenizer

# **Load the model and test**

In [44]:
# import torch
# from tqdm.auto import tqdm

# def evaluate_model(model, encode_tokenizer, decode_tokenizer, dataset, device):
#     model.eval()

#     predictions, references = [], []
#     for example in tqdm(dataset, desc="Translating"):
#         with encode_tokenizer.as_target_tokenizer():
#             input_ids = encode_tokenizer.encode(example['Bahnar'], return_tensors='pt').to(device)

#         output_ids = model.generate(input_ids)[0]
#         pred = decode_tokenizer.decode(output_ids, skip_special_tokens=True)
#         predictions.append([pred])

#         references.append([example['English']])

#     return predictions, references


In [45]:
# model_path = '/kaggle/working/model1.1.1'
# tokenizer_path = '/kaggle/working/model1.1.1'

# # Load the model and tokenizer
# loaded_model = AutoModelForSeq2SeqLM.from_pretrained(model_path).to(device)
# loaded_tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)

# # Evaluate the model
# test_examples = [{'Bahnar': ex['Bahnar'], 'English': ex['English']} for ex in ba_en_test_dataset]
# predictions, references = evaluate_model(loaded_model, en_ba_tokenizer, en_ba_tokenizer, test_examples, device)

In [46]:
# preds = []
# refs = []
# for pred in predictions:
#     preds.append(pred[0])

# for ref in references:
#     refs.append(ref)

# print(refs[:10])
# print(preds[:10])

In [47]:
# # Calculate and print the BLEU score
# bleu = sacrebleu.corpus_bleu(preds, refs)
# print("BLEU: ", round(bleu.score, 2))

# # Calculate CHRF
# chrf = sacrebleu.corpus_chrf(preds, refs)
# print("CHRF:", round(chrf.score, 2))

# # Calculate TER
# metric = sacrebleu.metrics.TER()
# ter = metric.corpus_score(preds, refs)
# print("TER:", round(ter.score, 2))